In [1]:
import json
import math
import os
import numpy as np
import scipy
import scipy.stats
from functools import reduce

In [2]:
snapshot_paths={}
data={}
for exp_folder in os.listdir():
    if 'recoverability' in exp_folder:
        print(exp_folder)
        path=exp_folder
        size=int(exp_folder.split('_')[1]) # Chord Size (4,8,16)
        for i in os.listdir(path):
            path_iter=os.path.join(exp_folder,i)
            iteration=int(i.split('_')[1]) # Iteration
#             print(i)
            for f_name in os.listdir(path_iter):
                if '.json' in f_name:
                    f_path=os.path.join(path_iter,f_name)
#                     print(f_path)
                elif 'backup' in f_name:
                    backup_path=os.path.join(path_iter,f_name)
                    t = int(f_name.split('_')[2]) # Timestamp
                    snapshot_paths[(size,iteration,t)] = backup_path
#                     for backup_name in os.listdir(backup_path):
#                         if '.json' in f_name:
#                             f_path=os.path.join(backup_path,f_name)
#                             print(f_path)

for b in sorted(snapshot_paths, key = lambda x: (x[0],x[1], x[2])):
    path=snapshot_paths[b]
    for f_name in os.listdir(path):
#         print(os.path.join(path,f_name))
        hostname=f_name.split('_')[0]
        host_key=(b[0],b[1],hostname)
        try:
            with open(os.path.join(path,f_name)) as f:
                host_snapshot=json.load(f)
                if host_key in data:
                    if host_snapshot!=data[host_key][-1][0]:
#                         print(f'Old => {data[host_key][-1][0]}')
#                         print(f'New => {host_snapshot}')
                        data[host_key].append((host_snapshot,b[2]))
                else:
                    #host_snapshot['t']=b[2]
                    data[host_key]=[(host_snapshot,b[2])]
        except ValueError:
            pass

recoverability_16
recoverability_32


In [19]:
signal_data={}
crash_times={}
recover_times={}
for exp_folder in os.listdir():
    if 'recoverability' in exp_folder:
        print(exp_folder)
        path=exp_folder
        size=int(exp_folder.split('_')[1]) # Chord Size (4,8,16)
        for i in os.listdir(path):
            path_iter=os.path.join(exp_folder,i)
            iteration=int(i.split('_')[1]) # Iteration
            with open(os.path.join(path_iter,'actions.log')) as f:
                for line in f:
                    if 'Stop' in line:
                        t=int(line.split()[-1])
                        node=line.split()[1]
                        signal_data[(size,iteration,node)]={'crash':t}
                        if (size,iteration) not in crash_times or t < crash_times[(size,iteration)]:
                            crash_times[(size,iteration)]=t
                    else:
                        t=int(line.split()[-1])
                        node=line.split()[1]
                        signal_data[(size,iteration,node)]['recover']=t
                        if (size,iteration) not in recover_times or t < recover_times[(size,iteration)]:
                            recover_times[(size,iteration)]=t

recoverability_16
recoverability_32


In [20]:
print(crash_times)

{(16, 0): 1640928239, (16, 1): 1640928577, (16, 10): 1640931618, (16, 11): 1640931956, (16, 12): 1640932294, (16, 13): 1640932632, (16, 14): 1640932970, (16, 15): 1640933308, (16, 16): 1640933646, (16, 17): 1640933984, (16, 18): 1640934321, (16, 19): 1640934659, (16, 2): 1640928915, (16, 3): 1640929253, (16, 4): 1640929591, (16, 5): 1640929929, (16, 6): 1640930267, (16, 7): 1640930604, (16, 8): 1640930942, (16, 9): 1640931280, (32, 0): 1640955354, (32, 1): 1640955698, (32, 10): 1640958787, (32, 11): 1640959130, (32, 12): 1640959473, (32, 13): 1640959816, (32, 14): 1640960159, (32, 15): 1640960502, (32, 16): 1640960846, (32, 17): 1640961188, (32, 18): 1640961532, (32, 19): 1640961875, (32, 2): 1640956041, (32, 3): 1640956384, (32, 4): 1640956728, (32, 5): 1640957071, (32, 6): 1640957414, (32, 7): 1640957757, (32, 8): 1640958101, (32, 9): 1640958444}


In [34]:
exp_2_ops_data={16:{
    'live':{'pre-crash':[],'during-crash':[],'after-crash':[],},
    'crash':{'pre-crash':[],'after-crash':[],}},
                32:{
    'live':{'pre-crash':[],'during-crash':[],'after-crash':[],},
    'crash':{'pre-crash':[],'after-crash':[],}},}

exp_2_time_data={16:{
    'live':{'pre-crash':[],'during-crash':[],'after-crash':[],},
    'crash':{'pre-crash':[],'after-crash':[],}},
                32:{
    'live':{'pre-crash':[],'during-crash':[],'after-crash':[],},
    'crash':{'pre-crash':[],'after-crash':[],}},}

# crash_times=
# exp_2_time_data={4:[],8:[],16:[],}
# for size in [4,8,16]:
#     for i in range(0,20):
#         exp_1_ops_data[size][i]=[]
#         exp_1_time_data[size][i]=[]
#         print(size,i)
# print(exp_1_ops_data)
for x in data:
    ops=len(data[x])-1
    #diff=data[x][-1][1]-data[x][0][1]
    first_op_t=data[x][0][1]
    ops_pc=0 # Pre-crash operation
    ops_dc=0 # During-crash operation
    ops_ac=0 # After crash operation
    last_op_t_pc=0 # Last operation time Pre-crash
    last_op_t_dc=0 # Last operation time During-crash
    last_op_t_ac=0 # Last operation time After crash
    alive = 'live' if x not in signal_data else 'crash' # Whether a crash node or alive node
    relative_crash_time=crash_times[(x[0],x[1])]
    relative_recover_time=recover_times[(x[0],x[1])]
    for event in data[x][1:]:
        if event[1] < relative_crash_time: # Pre-crash operation
            ops_pc+=1
            last_op_t_pc=event[1]
        elif event[1] < relative_recover_time: # During-crash operation
            ops_dc+=1
            last_op_t_dc=event[1]
        else: # After crash operation
            ops_ac+=1
            last_op_t_ac=event[1]                
    exp_2_ops_data[x[0]][alive]['pre-crash'].append(ops_pc)
    if alive == 'live':
        exp_2_ops_data[x[0]][alive]['during-crash'].append(ops_dc)
    exp_2_ops_data[x[0]][alive]['after-crash'].append(ops_ac)
    exp_2_time_data[x[0]][alive]['pre-crash'].append(last_op_t_pc-first_op_t if last_op_t_pc and last_op_t_pc>first_op_t else 0)
    if alive == 'live':
        exp_2_time_data[x[0]][alive]['during-crash'].append(last_op_t_dc-relative_crash_time if last_op_t_dc and last_op_t_dc>relative_crash_time else 0)
    exp_2_time_data[x[0]][alive]['after-crash'].append(last_op_t_ac-relative_recover_time if last_op_t_ac and last_op_t_ac>relative_recover_time else 0)

        

In [35]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m

def bound_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return h



In [36]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    name=f'Pre-Crash Live Node',
    x=['16 Nodes','32 Nodes'], y=[mean_confidence_interval(exp_2_ops_data[i]['live']['pre-crash']) for i in [16,32]],
    error_y=dict(type='data', array=[bound_confidence_interval(exp_2_ops_data[i]['live']['pre-crash']) for i in [16,32]])
))
fig.add_trace(go.Bar(
    name=f'During-Crash Live Node',
    x=['16 Nodes','32 Nodes'], y=[mean_confidence_interval(exp_2_ops_data[i]['live']['during-crash']) for i in [16,32]],
    error_y=dict(type='data', array=[bound_confidence_interval(exp_2_ops_data[i]['live']['during-crash']) for i in [16,32]])
))
fig.add_trace(go.Bar(
    name=f'After-Crash Live Node',
    x=['16 Nodes','32 Nodes'], y=[mean_confidence_interval(exp_2_ops_data[i]['live']['after-crash']) for i in [16,32]],
    error_y=dict(type='data', array=[bound_confidence_interval(exp_2_ops_data[i]['live']['after-crash']) for i in [16,32]])
))
fig.add_trace(go.Bar(
    name=f'Pre-Crash Crash Node',
    x=['16 Nodes','32 Nodes'], y=[mean_confidence_interval(exp_2_ops_data[i]['crash']['pre-crash']) for i in [16,32]],
    error_y=dict(type='data', array=[bound_confidence_interval(exp_2_ops_data[i]['crash']['pre-crash']) for i in [16,32]])
))
fig.add_trace(go.Bar(
    name=f'After-Crash Crash Node',
    x=['16 Nodes','32 Nodes'], y=[mean_confidence_interval(exp_2_ops_data[i]['crash']['after-crash']) for i in [16,32]],
    error_y=dict(type='data', array=[bound_confidence_interval(exp_2_ops_data[i]['crash']['after-crash']) for i in [16,32]])
))
fig.update_layout(title="Number of Operations made to stabilize a Chord ring with N nodes in Pre-crash and After-Crash Intervals",barmode='group',yaxis_title="Stabilization(s)",)
fig.show()

In [37]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    name=f'Pre-Crash Live Node',
    x=['16 Nodes','32 Nodes'], y=[mean_confidence_interval(exp_2_time_data[i]['live']['pre-crash']) for i in [16,32]],
    error_y=dict(type='data', array=[bound_confidence_interval(exp_2_time_data[i]['live']['pre-crash']) for i in [16,32]])
))
fig.add_trace(go.Bar(
    name=f'During-Crash Live Node',
    x=['16 Nodes','32 Nodes'], y=[mean_confidence_interval(exp_2_time_data[i]['live']['during-crash']) for i in [16,32]],
    error_y=dict(type='data', array=[bound_confidence_interval(exp_2_time_data[i]['live']['during-crash']) for i in [16,32]])
))
fig.add_trace(go.Bar(
    name=f'After-Crash Live Node',
    x=['16 Nodes','32 Nodes'], y=[mean_confidence_interval(exp_2_time_data[i]['live']['after-crash']) for i in [16,32]],
    error_y=dict(type='data', array=[bound_confidence_interval(exp_2_time_data[i]['live']['after-crash']) for i in [16,32]])
))
fig.add_trace(go.Bar(
    name=f'Pre-Crash Crash Node',
    x=['16 Nodes','32 Nodes'], y=[mean_confidence_interval(exp_2_time_data[i]['crash']['pre-crash']) for i in [16,32]],
    error_y=dict(type='data', array=[bound_confidence_interval(exp_2_time_data[i]['crash']['pre-crash']) for i in [16,32]])
))
fig.add_trace(go.Bar(
    name=f'After-Crash Crash Node',
    x=['16 Nodes','32 Nodes'], y=[mean_confidence_interval(exp_2_time_data[i]['crash']['after-crash']) for i in [16,32]],
    error_y=dict(type='data', array=[bound_confidence_interval(exp_2_time_data[i]['crash']['after-crash']) for i in [16,32]])
))
fig.update_layout(title="Stabilization time (s) for Chord ring with N nodes in Pre-crash and After-Crash Intervals",barmode='group',yaxis_title="Stabilization(s)",)
fig.show()